# Intro to julia

Reference: [The Julia Documentation](https://docs.julialang.org)

This lecture gives an overview of Julia. [The Julia–Matlab–Python Cheatsheet](https://cheatsheets.quantecon.org)
Is a valuable resource for translating between the three languages.

## Integers

Julia uses a math-like syntax for manipulating integers:

In [1]:
1 + 1 # Addition

2

In [2]:
2 * 3 # Multiplication

6

In [3]:
x = 5; # semicolon is optional but supresses output if used in last line
x^2 # Powers

25

In Julia everything has a type. This is similar in spirit to 
a class in Python, but much more lightweight. 
An integer defaults to type `Int`, 
which is either 32-bit (`Int32`) or 64-bit (`Int64`) depending
on the processor of the machine. 
These are "primative type", instances of the type are stored in memory as 
a fixed length sequence of bits.
 
We can see the bits using the function `bitstring`:

In [4]:
bitstring(1)

"0000000000000000000000000000000000000000000000000000000000000001"

The first bit is a sign bit: if it is equal to `1` the 
integer is negative

In [5]:
bitstring(-1)

"1111111111111111111111111111111111111111111111111111111111111111"

This may be counter-intuitive (did you expect `1000…00001`?)
but has the nice benefit that addition behaves the same for 
positive integers and negative integers.  

`Int` follows [modular arithmetic](https://en.wikipedia.org/wiki/Modular_arithmetic),
that is, it is equivalent to the ring of integers modulo `2^p` where `p` is `32` or `64`. 
Thus for $p = 64$ we are interpreting
$2^{32} \mod 2^{64}$ through $(2^{64}-1) \mod 2^{64}$ as negative numbers but 
they are not treated specially.


**Example (addition)**
Consider `(-1) + 1`. They behave like modular arithmetic so we have:
$$
(-1 \mod 2^p) + (1 \mod 2^p) = (2^p-1 \mod 2^p) + (1 \mod 2^p) = 2^p \mod 2^p = 0 \mod 2^p
$$

**Example (multiplication)**
Consider `(-2) * 2`. We have:
$$
(-2 \mod 2^p) * (2 \mod 2^p) = (2^p-2 \mod 2^p) * (2 \mod 2^p) = (2^(p+1)-4 \mod 2^p) = -4 \mod 2^p
$$




We can find the largest and smallest instances of a type using `typemax` and `typemin`:

In [6]:
typemax(Int32)

2147483647

In [7]:
bitstring(typemax(Int32))

"01111111111111111111111111111111"

In [8]:
typemin(Int32)

-2147483648

In [9]:
bitstring(typemin(Int32))

"10000000000000000000000000000000"

Since `Int64` behaves like modulo arithmetic we do not "overflow" but rather wrap around:

In [10]:
typemax(Int64) + 1 # returns typemin(Int32)

-9223372036854775808

There are other primitive integer types: `Int8` and `Int16` are like `Int32` and `Int64` but with
fewer bits. `UInt8`, `UInt16`, `UInt32`, and `UInt64` are unsigned integers, e.g., we do not interpret
the number as negative if the first bit is `1`. A non-primitive type is `BigInt` which allows arbitrary length
integers, which we can create using `big`:

In [11]:
x = typemax(Int64) + big(1) # Too big to be an `Int64`

9223372036854775808

In [12]:
x^100

3082994025277634745700106821545665721371798533305697458855342277921093731984476404705966539412410898240561729912372038501228893141921080152404642393776599077294434061519905424124601394226943601430916434383714716724720227331596950613701661034548948388721097667275438763758128508403297199458260277707301202460980093818414167080563342761482395862435185093942443540722363151770022221783243959592531336062998494209914752408019060720805124534382646051093613814848646062038662423487504326044361203708430489305864234333801401547140023376295718383390360728662900230671437151716615826286842267917560749586018165739492101920429719261285640125596833063891562865262157026023955919873792846823095854484520920509345944712871675691790827690907778485058829248588945681685288179787963931181062068092463984296225973082494056307958089189726701678735576365394146232076917088075949053636690459581128773097212746967276496496010810878000638239143750075543163240049874489986642327436441234458040254480825038220479904594615300

Note the number of bits is not fixed so it is possible to run out of memory if a number is 
astronomically large: go ahead and try `x^x`.

In addition to `+`, `-`, and `*` we have integer division `÷`, which rounds down:

In [13]:
5 ÷ 2 # equivalent to div(5,2)

2

Standard division `/` (or `\` for division on the right) creates a floating point number, which will be discussed in
the next chapter:

In [14]:
5 / 2 # alternatively 2 \ 5

2.5

## String

We have seen that `bitstring` returns a string of bits.
Strings can be created with quotation marks

In [15]:
str="hello world 😀"

"hello world 😀"

We can access characters of a string with brackets:

In [16]:
str[1],str[13]

('h', '😀')

Each character is a bit type, in this case using 32 bits/8 bytes:

In [17]:
typeof(str[6]), length(bitstring(str[6]))

(Char, 32)

Strings are not bit types, but rather point to the start of sequence 
of `Char` in memory.  In this case, there are $32*13=416$ bits/52 
bytes in memory

## Vector

We can create a vector using brackets:

In [18]:
v = [11,24,32]

3-element Vector{Int64}:
 11
 24
 32

Like a string, elements are accessed via brackets. Julia
uses 1-based indexing (like Matlab and Mathematica, unlike
Python and C which use 0-based indexing):

In [19]:
v[1],v[3]

(11, 32)

Accessing outside the range gives an error:

In [20]:
v[4]

LoadError: BoundsError: attempt to access 3-element Vector{Int64} at index [4]

Vectors can be made with different types, for example, 
here is a vector of three 8-bit integers:

In [21]:
v = [Int8(11), Int8(24), Int8(32)]

3-element Vector{Int8}:
 11
 24
 32

Just like strings, Vectors are not bit types, 
but rather point to the start of sequence of the corresponding type.  
In this last case, there are $3*8=24$ bits/3 bytes in memory.

## Parsing strings

We can use the command `parse` to turn a string into an integer:

In [22]:
parse(Int, "123")

123

We can specify base 2 as an optional argument. If we are specifying
bits its safer to parse as a `UInt32`, otherwise the first bit
is not recognised as a sign:

In [23]:
bts = "11110000100111111001100110001010"
x = parse(UInt32, bts; base=2)

0xf09f998a

The function `reinterpret` allows us to reinterpret the resulting 
sequence of 32 bits as a different type. For example, we can reinterpret
as an `Int32` in which case the first bit is taken to be the sign bit
and we get a negative number:

In [24]:
reinterpret(Int32, x)

-257975926

We can also reinterpret as a `Char`:

In [25]:
reinterpret(Char, x)

'🙊': Unicode U+1F64A (category So: Symbol, other)

We will use `parse` and `reinterpret` in problem sheets and the midterm exam
as it allows one to easily manipulate bits. This is not actually how one should
do it as it is slow.

## Bitwise operations (advanced)

In practice, one should manipulate bits using bitwise operations. 
These will not be required in this course and are not examinable, but
are valuable to know if you have a career involving high performance computing.
The `p << k` shifts the bits of `p` to the left `k` times inserting zeros, 
while `p >> k` shifts to the right:

In [26]:
print(bitstring(23));
print(bitstring(23 << 2));
print(bitstring(23 >> 2));

000000000000000000000000000000000000000000000000000000000001011100000000000000000000000000000000000000000000000000000000010111000000000000000000000000000000000000000000000000000000000000000101

The operations `&`, `|` and `⊻` do bitwise and, or, and xor.